# Person Directory

## Objective

 This notebook demonstrates how to identify faces in an image against a known set of persons. It begins by building a Person Directory, where each subfolder in a specified directory represents an individual. For each subfolder, a person is created and all face images within it are enrolled to that person.

| Enrollment | Searching |
| :-: | :-: |
| ![PD_enrollment.png](https://media.githubusercontent.com/media/Azure-Samples/azure-ai-content-understanding-python/refs/heads/zhizho/face/data/face/PD_enrollment.png) | ![PD_searching.png](https://media.githubusercontent.com/media/Azure-Samples/azure-ai-content-understanding-python/refs/heads/zhizho/face/data/face/PD_searching.png) |

## Create Azure content understanding face client
> The [AzureContentUnderstandingFaceClient](../python/content_understanding_face_client.py) is a utility class for interacting with the Content Understanding Face service. Before the official SDK is released, this acts as a lightweight SDK. Set the constants **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, and **AZURE_AI_API_KEY** with your Azure AI Service information.

> ⚠️ Important:
You must update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
If you skip this step, the sample may not run correctly.

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is much safer and is highly recommended for production environments.

In [ ]:
import logging
import os
import uuid
import sys
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.identity.aio import DefaultAzureCredential
from azure.ai.contentunderstanding.aio import ContentUnderstandingClient
from azure.ai.contentunderstanding.models import PersonDirectory, FaceSource
from sample_helper import (
    read_image_to_base64,
    read_image_to_base64_bytes
)

# Add the parent directory to the Python path to import the sample_helper module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'python'))

load_dotenv()
logging.basicConfig(level=logging.INFO)

endpoint = os.environ.get("AZURE_CONTENT_UNDERSTANDING_ENDPOINT")
# Return AzureKeyCredential if AZURE_CONTENT_UNDERSTANDING_KEY is set, otherwise DefaultAzureCredential
key = os.getenv("AZURE_CONTENT_UNDERSTANDING_KEY")
credential = AzureKeyCredential(key) if key else DefaultAzureCredential()
# Create the ContentUnderstandingClient
client = ContentUnderstandingClient(endpoint=endpoint, credential=credential)

## Build a Person Directory

In [ ]:
folder_path = "../data/face/enrollment_data"  # Replace with the path to your folder containing subfolders of images

# Create a person directory
person_directory_id = f"person_directory_id_{uuid.uuid4().hex[:8]}"

# Create a person directory first
print(f"🔧 Creating person directory '{person_directory_id}'...")

person_directory = PersonDirectory(
            description=f"Sample person directory for delete person demo: {person_directory_id}",
            tags={"demo_type": "delete_person"},
        )
person_directory = await client.person_directories.create(person_directory_id, resource=person_directory)
logging.info(f"Created person directory with ID: {person_directory_id}")

# Initialize persons list
persons: list = []

# Iterate through all subfolders in the folder_path
for subfolder_name in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder_name)
    if os.path.isdir(subfolder_path):
        person_name = subfolder_name
        # Add a person for each subfolder
        person = await client.person_directories.add_person(person_directory_id, tags={"name": person_name})
        print(f"🔧 Creating person '{person_name}'...")
        logging.info(f"Created person {person_name} with person_id: {person['personId']}")
        if person:
            # Initialize person entry in persons list
            person_entry = {
                'personId': person['personId'],
                'name': person_name,
                'faceIds': []
            }

            # Iterate through all images in the subfolder
            for filename in os.listdir(subfolder_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(subfolder_path, filename)
                    # Convert image to base64
                    image_data = read_image_to_base64(image_path)
                    # Add a face to the Person Directory and associate it to the added person
                    print(f"🔧 Adding face from image '{image_path}' to person '{person_name}'...")
                    print(f"Image Data: ", image_data)
                    face = await client.person_directories.add_face(
                        person_directory_id=person_directory_id, 
                        body={
                            "faceSource": { "data": image_data },
                            "personId": person['personId']
                        }
                    )
                    if face:
                        logging.info(f"Added face from {filename} with face_id: {face['faceId']} to person_id: {person['personId']}")
                    else:
                        logging.warning(f"Failed to add face from {filename} to person_id: {person['personId']}")

            # Add person entry to persons list
            persons.append(person_entry)

logging.info("Done")
logging.info(f"Created {len(persons)} persons:")
for person in persons:
    logging.info(f"Person: {person['name']} (ID: {person['personId']}) with {len(person['faceIds'])} faces")

### Identifying person
Detect multiple faces in an image and identify each one by matching it against enrolled persons in the Person Directory.

In [ ]:
test_image_path = "../data/face/family.jpg"  # Path to the test image

# Detect faces in the test image
image_data = read_image_to_base64(test_image_path)
detected_faces = await client.faces.detect(data=image_data)
for face in detected_faces['detectedFaces']:
    identified_persons = await client.person_directories.identify_person(person_directory_id, image_data, face['boundingBox'])
    if identified_persons.get("personCandidates"):
        person = identified_persons["personCandidates"][0]
        name = person.get("tags", {}).get("name", "Unknown")
        logging.info(f"Detected person: {name} with confidence: {person.get('confidence', 0)} at bounding box: {face['boundingBox']}")

logging.info("Done")

### Adding and associating a new face
You can add a new face to the Person Directory and associate it with an existing person.

In [ ]:
person_bill = next(person for person in persons if person['name'] == 'Bill')
new_face_image_path = "../data/face/new_face_image.jpg" # The path to the face image you want to add.
existing_person_id = person_bill['personId'] # The unique ID of the person to whom the face should be associated.

# Convert the new face image to base64
image_data = read_image_to_base64(new_face_image_path)
# Add the new face to the person directory and associate it with the existing person
face = await client.person_directories.add_face(person_directory_id, image_data, existing_person_id)
if face:
    logging.info(f"Added face from {new_face_image_path} with face_id: {face['faceId']} to person_id: {existing_person_id}")
else:
    logging.warning(f"Failed to add face from {new_face_image_path} to person_id: {existing_person_id}")

### Associating a list of already enrolled faces

You can associate a list of already enrolled faces in the Person Directory with their respective persons. This is useful if you have existing face IDs to link to specific persons.

In [ ]:
existing_person_id = person_bill['personId']  # The unique ID of the person to whom the face should be associated.
existing_face_id_list = [person_bill['faceIds'][0], person_bill['faceIds'][1], person_bill['faceIds'][2]]  # The list of face IDs to be associated.

# Associate the existing face IDs with the existing person
await client.person_directories.update_person(person_directory_id, existing_person_id, face_ids=existing_face_id_list)

### Associating and disassociating a face from a person
You can associate or disassociate a face from a person in the Person Directory. Associating a face links it to a specific person, while disassociating removes this link.

In [ ]:
person_mary = next(person for person in persons if person['name'] == 'Mary')
existing_face_id = person_mary['faceIds'][0]  # The unique ID of the face.

# Remove the association of the existing face ID from the person
await client.person_directories.update_face(person_directory_id, existing_face_id, person_id="") # The person_id is set to "" to remove the association
logging.info(f"Removed association of face_id: {existing_face_id} from the existing person_id")
logging.info(await client.person_directories.get_face(person_directory_id, existing_face_id)) # This will return the face information without the person association

# Associate the existing face ID with a person
existing_person_id = "existing_person_id"  # The unique ID of the person to be associated with the face.
await client.person_directories.update_face(person_directory_id, existing_face_id, person_id=existing_person_id)
logging.info(f"Associated face_id: {existing_face_id} with person_id: {existing_person_id}")
logging.info(await client.person_directories.get_face(person_directory_id, existing_face_id)) # This will return the face information with the new person association

### Updating metadata (tags and descriptions)
You can add or update tags for individual persons, and both descriptions and tags for the Person Directory. These metadata fields help organize, filter, and manage your directory.

In [ ]:
# Update the description and tags for the Person Directory
person_directory_description = "This is a sample person directory for managing faces."
person_directory_tags = {"project": "face_management", "version": "1.0"}

await client.person_directories.update(
    person_directory_id,
    description=person_directory_description,
    tags=person_directory_tags
)
logging.info(f"Updated Person Directory with description: '{person_directory_description}' and tags: {person_directory_tags}")
logging.info(await client.person_directories.get(person_directory_id)) # This will return the updated person directory information

# Update the tags for an individual person
existing_person_id = "existing_person_id"  # The unique ID of the person to update.
person_tags = {"role": "tester", "department": "engineering", "name": ""} # This will remove the name tag from the person.

await client.person_directories.update_person(
    person_directory_id,
    existing_person_id,
    tags=person_tags
)
logging.info(f"Updated person with person_id: {existing_person_id} with tags: {person_tags}")
logging.info(await client.person_directories.get_person(person_directory_id, existing_person_id)) # This will return the updated person information

### Deleting a face
You can also delete a specific face. Once the face is deleted, the association between the face and its associated person is removed.

In [ ]:
existing_face_id = person_mary['faceIds'][0] # The unique ID of the face to delete.

await client.person_directories.delete_face(person_directory_id, existing_face_id)
logging.info(f"Deleted face with face_id: {existing_face_id}")

### Deleting a person

When a person is deleted from the Person Directory, all the faces associated with that person remain in the Person Directory, but the association between the person and the faces is removed. This means the faces are no longer associated to any person in the Person Directory.

In [ ]:
existing_person_id = person_mary['personId']  # The unique ID of the person to delete.

await client.person_directories.delete_person(person_directory_id, existing_person_id)
logging.info(f"Deleted person with person_id: {existing_person_id}")

### Deleting a person and their associated faces

To completely remove a person and all their associated faces from the Person Directory, you can delete the person along with their face associations. This operation ensures that no residual data related to the person remains in the directory.

In [ ]:
existing_person_id = person_bill['personId']  # The unique ID of the person to delete.

# Get the list of face IDs associated with the person
response = await client.person_directories.get_person(person_directory_id, existing_person_id)
face_ids = response.get('faceIds', [])

# Delete each face associated with the person
for face_id in face_ids:
    logging.info(f"Deleting face with face_id: {face_id} from person_id: {existing_person_id}")
    await client.person_directories.delete_face(person_directory_id, face_id)

# Delete the person after deleting all associated faces
await client.person_directories.delete_person(person_directory_id, existing_person_id)
logging.info(f"Deleted person with person_id: {existing_person_id} and all associated faces.")